In [ ]:
from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
import numpy  as np

import subprocess
import requests
import random
import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

default_path    = os.path.join(set_default_path() ,'Production_Environment')
server_path     = os.path.join(default_path       ,'Servers') 
collectors_path = os.path.join(server_path        ,'Collectors')

# Events
events_path     = os.path.join(collectors_path    ,'Events')
data_path       = os.path.join(events_path        ,'Data')
File_path       = os.path.join(data_path          ,'Final_Events.csv')

# User Base
UserBase_path  = os.path.join(collectors_path ,'UserBase')
user_data_path = os.path.join(UserBase_path   ,'Data')
user_data      = os.path.join(user_data_path  ,'User_data.json')

In [5]:
df = pd.read_csv(os.path.join(data_path,'Final_Events.csv'))

In [13]:
with open(user_data,mode = 'r') as f:
    users = json.load(f)

usernames = list(users.keys())
username = random.choice(usernames)
print(f'Sample username: {username}')
interactions = users[username]['CLICKED']

Sample username: zavalaeric


In [ ]:
# results  = []
# for idx in range(n_rows):
#     row = df.iloc[idx].fillna('')
#     item = {
#         'TITLE': row['Title'],
#         'DATE' : row['Start Date'],
#         'MAINPAGE' : row['Main Page'],
#         'ADDRESS1' : row['Address1'],
#         'ADDRESS2' : row['Address2'],
#         'WEHN' : row['When'],
#         'THUMB': row['thumb'],
#         'IMAGE' : row['image'],
#         'DESC' : row['Desc'],
#         'LongDesc': ''
#     }
#     results.append(item)


In [ ]:
url  = "http://127.0.0.1:5000/get_col"
select = {
    'COL': 'ID'
}
ids  = query(url = url, data = select, mode = 'POST')

In [23]:
url  = "http://127.0.0.1:5001/get_weights"
select = {
    'ITEMS': interactions
}
weights  = query(url = url, data = select, mode = 'POST')

In [ ]:
# Reorder in the same 
weights = [weights[str(i)] for i in ids]
selections = tuple(random.choices(population=ids,weights = weights, k = 5))

In [45]:
type(selections)

tuple

In [48]:
url  = "http://127.0.0.1:5000/get_rows"
select = {
    'IDS': selections
}
events_items  = query(url = url, data = select, mode = 'POST')

In [50]:
for item in events_items:
    print(item)

[25, 'Anime Milwaukee 2025', 'Mar 7', 'https://www.visitmilwaukee.org/event/anime-milwaukee-2025/10586/', 'Baird Center, 400 W Wisconsin Ave', 'Milwaukee, WI', 'Fri, Mar 7, 9 AM – Sun, Mar 9, 4 PM', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQsa4jGrPxbgFCGiFMK4E0nb5_jIKTAAK-083DNrks9ENVKVALJDJKKGEI&s', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQLj8J3LVFsIcVs8a-AsnJ-aEEUqMK4UKVfuK4giklbAQ&s=10', "Anime Milwaukee (AMKE) is Wisconsin's largest anime convention, celebrating Japanese animation, culture, and fandom. The upcoming event will take place from March 7 to 9, 2025, at the Baird Center...", '']
[62, 'Isekai Anime Con', 'Mar 22', 'https://www.cityspark.com/html5-calendar/details/isekai-anime-con/15170443/2025-03-22T00', 'Mountain America Exposition Center, 9575 State St', 'Sandy, UT', 'Sat, Mar 22', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRxXyrJ3X20OamC793Jk-i5_T2aI6zyQPPJQQZaw4-9XrHyb2gQfC91PXg&s', 'https://encrypted-tbn0.gstatic.com/images?q=

---

In [131]:
url  = "http://127.0.0.1:5004/item"
select = {
    'USER_ID': 'Something',
    'NUMBER': 2
}#
reccomendation = query(url = url, data = select, mode = 'POST')

In [132]:
reccomendation

[[429,
  'MomoCon 2025',
  'May 22',
  'https://animecons.com/events/info/24114/momocon-2025',
  'Georgia World Congress Center, 285 Andrew Young International Blvd NW',
  'Atlanta, GA',
  'Thu, May 22 – Sun, May 25',
  'https://www.google.com/maps/vt/data=i4iZQe1eEgJZ5P8ZppzS0Hp2zuxx1oqhkZ6LZ1ZRhn-oN3ha_0CzL5iN4qumS-Q3-GlXne6O7ZsOzcsKGF87EAzWi_fj5G1LLQ79AaJsiL10BxQ-NQs',
  '',
  'MomoCon brings together fans of Japanese Anime, American Animation, Comics, Video Games, and Tabletop Games to celebrate their passion by costuming / cosplay, browsing the huge exhibitors hall...',
  ''],
 [432,
  'Momocon - Atlanta, GA - May 22-25 — Tea & Absinthe',
  'May 22',
  'https://www.teaandabsinthe.com/event-calendar/2025/5/22/momocon-atlanta-ga-may-22-25',
  '',
  '',
  'Thu, May 22, 5 PM – Sun, May 25, 5 PM',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFkpuzM63ko-vSTRJcATdDUbDkcqN6MDq0ADKvPaeISkwRog7PW1Xhvg0&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYZkB0g_uyDumNV

In [ ]:
interactinos = list(sample_user.values())[0]['CLICKED']

In [74]:
# interactions